# input(나이와 좋아하는 웹툰) 입력
## 옵션 1: q를 입력받을 때까지 계속 받음, 3개 이상

+ 19세 이하일 경우 성인용 웹툰 data에서 삭제
+ 평점 상위 10%에는 가중치 높게, 하위 10%에는 가중치 낮게, 나머지는 동일

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("./naver_daum_integration.csv")

# 추천 점수 저장할 리스트 선언 (data의 idx(1~2383)을 인덱스로)
recommend_score = [0 for i in range(len(data)+1)]

# 나이 입력받기
age = input('나이를 입력하세요: ')

# 잘못 입력했을 경우 정확히 입력할 때까지 반복
while not age.isdigit():
    age = input('정확한 나이를 입력하세요: ')
    if age.isdigit():
        break
    
# 전체 웹툰 제목 받아오기 (input 값 비교용)
title = np.array(data['name'].tolist())

# 입력한 웹툰의 제목을 저장하는 리스트
input_list = []

# q를 입력할 때까지 좋아하는 웹툰 제목을 입력받음, 무조건 세 개 이상 입력해야함, 중복 입력 불가능
while 1:
    a = input('좋아하는 웹툰을 입력하세요: ')
    if a == 'q':
        if len(input_list) >= 3:
            break
        else:
            print("웹툰을 세 개 이상 입력하세요")
    elif a not in title:
        print('입력한 웹툰이 존재하지 않습니다.')
    elif a in input_list:
        print('중복된 웹툰입니다.')
    else:
        input_list.append(a)

# like_list에 입력받은 웹툰 정보를 저장, 각 요소는 Series type으로 되어있음
like_list = []
for i in input_list:
    like_list.append(data[data.name == i])

# like_list에 저장된 각 요소를 하나의 DataFrame으로 합쳐줌 -> like_df에는 입력받은 웹툰들이 DataFrame의 행 형태로 있음
like_df = like_list[0]
for i in like_list[1:]:
    like_df = pd.concat([like_df, i])

# 입력받은 나이가 19세 미만이면 19세 웹툰은 data(추천 데이터셋)에서 삭제
if int(age) < 19:
    data = data[data.age_limit != 1]
    

############ score 가중치 계산 ############

# score의 내림차순으로 정렬한 data를 score_arrange_data에 저장
score_arrange_data = data.sort_values(by=['score'], axis=0, ascending=False)

# data 개수의 10%는 238개
ten_percent = round(len(data) * 0.1)

# 상위 10%의 가장 낮은 점수(9.96)과 하위 10%의 가장 높은 점수(8.96)를 저장 (동점인 항목까지 가중치 부여에 포함하기 위함)
high_score = score_arrange_data.iloc[ten_percent - 1].score
low_score = score_arrange_data.iloc[len(data)-ten_percent].score

# score가 높은 idx(291개), score가 낮은 idx(238개)를 배열에 각각 저장
high_score_list = data[data['score'] >= high_score].idx.values
low_score_list = data[data['score'] <= low_score].idx.values

# score가 높은 웹툰과 낮은 웹툰에 해당하는 idx의 recommend score 점수에 가중치 부여 (각각 1과 -1)
for i in high_score_list:
    recommend_score[i] += 1
    
for i in low_score_list:
    recommend_score[i] -= 1

### 위의 코드는 실행할 때마다 값을 입력해야므로 바로 값을 확인할 수 있도록 만든 연습 코드
 - 나이와 좋아하는 웹툰 input이 바로 들어가있음

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("./naver_daum_integration.csv")
recommend_score = [0 for i in range(len(data)+1)]

age = 19
input_list = ['유미의 세포들', '무지개다리 파수꾼', '현세이의 보이는 라디오']

like_list = []
for i in input_list:
    like_list.append(data[data.name == i])
    
like_df = like_list[0]
for i in like_list[1:]:
    like_df = pd.concat([like_df, i])

if age < 19:
    data = data[data.age_limit != 1]
    
###### 수정된 score 계산 적용 #####

score_arrange_data = data.sort_values(by=['score'], axis=0, ascending=False)
ten_percent = round(len(data) * 0.1)

high_score = score_arrange_data.iloc[ten_percent - 1].score
low_score = score_arrange_data.iloc[len(data)-ten_percent].score

data.loc[data['score'] >= high_score, 'idx']

high_score_list = data[data['score'] >= high_score].idx.values
low_score_list = data[data['score'] <= low_score].idx.values

for i in high_score_list:
    recommend_score[i] += 1
    
for i in low_score_list:
    recommend_score[i] -= 1

## 옵션 2: 입력할 웹툰 개수를 먼저 입력함

In [ ]:
age = input('나이를 입력하세요: ')

likeNum = input('입력할 좋아하는 웹툰 개수를 입력하세요: ')

input_list = list()

for i in range(int(likeNum)):
    input_list.append(input('좋아하는 웹툰을 입력하세요: '))

## 추가 기능: 나이 입력하지 않았을 경우 오류 처리

In [ ]:
age = input('나이를 입력하세요: ')

while not age.isdigit():
    age = input('정확한 나이를 입력하세요: ')
    if age.isdigit():
        break
        
age

## 평점 필터링 초기 설정

 - 입력한 웹툰 중 평점이 8.5 이하인 웹툰이 있으면 평점 8.5 이하의 웹툰도 추천, 없으면 추천하지 않음

In [ ]:
# 추천 요소에 평점 고려
score_count = 0

# 좋아하는 웹툰 중 평점 8.5 이하인 웹툰이 있으면 평점 8.5 이하인 웹툰도 추천
# 없으면 평점 8.5 이하인 웹툰인 data에서 삭제
for i in like_list:
    if (i.score < 8.5).any():
        score_count += 1

if score_count == 0:
    data = data[data.score > 8.5 ]